In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [3]:
driver = webdriver.Chrome()

In [4]:
driver.get("http://proxy.uchicago.edu/login?url=https://global.factiva.com/en/sess/login.asp?XSID=S003Wv90Gb95DEs5DEnODYqMDEsND7yMHn0VrJoWXNG2TBX4cjBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQQAA")

In [11]:
search_form = driver.find_element(By.ID,"ftx")

In [13]:
search_form.get_property("value")

'Hew'